<a href="https://colab.research.google.com/github/lukabombala/skyhacks/blob/master/tasks/main/skyhacks58.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import load_model
from sklearn.model_selection import train_test_split
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import io
import numpy as np
import pandas as pd
import os
from google.colab import drive

Using TensorFlow backend.


In [0]:
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return 1-f1_val


In [0]:
def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
base_model_2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# add a global spatial average pooling layer
x2 = base_model_2.output
x2 = GlobalAveragePooling2D()(x2)
# let's add a fully-connected layer
x2 = Dense(1024, activation='relu')(x2)
state = Dense(4, activation='softmax')(x2)
condition = Dense(4, activation='softmax')(x2)

# this is the model we will train
model_2 = Model(inputs=base_model_2.input, outputs=[state, condition])














219062272/219055592 [==============================] - 2s 0us/step


In [0]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
stuff = Dense(53, activation='sigmoid')(x)
room = Dense(6, activation='softmax')(x)

# # this is the model we will train
model = Model(inputs=base_model.input, outputs=[stuff, room])

In [0]:
for layer in base_model.layers:
    layer.trainable = False
    

In [0]:
labels_task_1 = ['Bathroom', 'Bathroom cabinet', 'Bathroom sink', 'Bathtub', 'Bed', 'Bed frame',
                 'Bed sheet', 'Bedroom', 'Cabinetry', 'Ceiling', 'Chair', 'Chandelier', 'Chest of drawers',
                 'Coffee table', 'Couch', 'Countertop', 'Cupboard', 'Curtain', 'Dining room', 'Door', 'Drawer',
                 'Facade', 'Fireplace', 'Floor', 'Furniture', 'Grass', 'Hardwood', 'House', 'Kitchen',
                 'Kitchen & dining room table', 'Kitchen stove', 'Living room', 'Mattress', 'Nightstand',
                 'Plumbing fixture', 'Property', 'Real estate', 'Refrigerator', 'Roof', 'Room', 'Rural area',
                 'Shower', 'Sink', 'Sky', 'Table', 'Tablecloth', 'Tap', 'Tile', 'Toilet', 'Tree', 'Urban area',
                 'Wall', 'Window']

labels_task2 = ['bathroom', 'bedroom', 'dinning_room', 'house', 'kitchen', 'living_room']

labels_task3 = [1,2,3,4]

root = os.path.join(os.getcwd(),"gdrive","My Drive","skyhacks")
rootdir = os.path.join(root, "resizeduni/")

imgcsv = pd.read_csv(os.path.join(root,"labels.csv")) #csv with image info
imgcsv2 = imgcsv[imgcsv["task2_class"]!="validation"] #csv with image info without images with "validation" room class

targets = imgcsv2.drop(["filename", "standard","task2_class", "tech_cond"], axis = 1).to_numpy()
targets2 = pd.get_dummies(imgcsv2, columns = ["task2_class"])[["task2_class_bathroom","task2_class_bedroom",
                                                               "task2_class_dining_room","task2_class_house","task2_class_kitchen","task2_class_living_room"]].to_numpy()
targets3 = to_categorical(imgcsv.drop(["filename", "task2_class", "tech_cond"]+labels_task_1, axis = 1).to_numpy()-1)
targets4 = to_categorical(imgcsv.drop(["filename", "task2_class", "standard"]+labels_task_1, axis = 1).to_numpy()-1)



In [0]:
imgset = []

for name in imgcsv2["filename"]:
    imgset.append(io.imread(os.path.join(rootdir, name))/255)
    
dataset = np.array([np.array(imgset[i]) for i in range(len(imgset))])    

In [0]:
imgset_2 = []

for name in imgcsv["filename"]:
    imgset_2.append(io.imread(os.path.join(rootdir, name))/255)
    
dataset_2 = np.array([np.array(imgset_2[i]) for i in range(len(imgset_2))])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dataset, targets, test_size=0.2, random_state = 42)
X_train, X_test, y_train2, y_test2 = train_test_split(dataset, targets2, test_size = 0.2, random_state = 42)

EarlyStop = EarlyStopping(monitor= "val_dense_6_f1", patience = 5, restore_best_weights=True)

losses = {
	"dense_5": "binary_crossentropy",
	"dense_6": "categorical_crossentropy",
}

model.compile(optimizer='adam', loss=losses, metrics = [f1])

model.fit(X_train,
 	{"dense_5": y_train, "dense_6": y_train2},
	validation_data=(X_test,
		{"dense_5": y_test, "dense_6": y_test2}), epochs=50, callbacks = [EarlyStop], batch_size = 16)

Train on 1752 samples, validate on 438 samples
Epoch 1/50
1752/1752 [==============================] - 57s 33ms/step - loss: 1.5976 - dense_5_loss: 0.2541 - dense_6_loss: 1.3435 - dense_5_f1: 0.4628 - dense_6_f1: 0.6167 - val_loss: 0.9690 - val_dense_5_loss: 0.2352 - val_dense_6_loss: 0.7338 - val_dense_5_f1: 0.3621 - val_dense_6_f1: 0.2369
Epoch 2/50
1752/1752 [==============================] - 37s 21ms/step - loss: 1.1061 - dense_5_loss: 0.2008 - dense_6_loss: 0.9053 - dense_5_f1: 0.3866 - dense_6_f1: 0.3854 - val_loss: 1.2523 - val_dense_5_loss: 0.2350 - val_dense_6_loss: 1.0173 - val_dense_5_f1: 0.2963 - val_dense_6_f1: 0.2609
Epoch 3/50
1752/1752 [==============================] - 38s 22ms/step - loss: 1.0423 - dense_5_loss: 0.1888 - dense_6_loss: 0.8535 - dense_5_f1: 0.3535 - dense_6_f1: 0.3379 - val_loss: 0.9716 - val_dense_5_loss: 0.2224 - val_dense_6_loss: 0.7491 - val_dense_5_f1: 0.2922 - val_dense_6_f1: 0.2228
Epoch 4/50
1752/1752 [==============================] - 38s 22ms/

In [0]:
X_train_2, X_test_2, y_train3, y_test3 = train_test_split(dataset_2, targets3, test_size=0.2, random_state = 42)
X_train_2, X_test_2, y_train4, y_test4 = train_test_split(dataset_2, targets4, test_size = 0.2, random_state = 42)

class_weight = {
    1: 1,
    2: 1,
    3: 11,
    4: 7
}

EarlyStop2 = EarlyStopping(monitor= "val_dense_3_f1", patience = 6, restore_best_weights=True)

losses_2 = {
	"dense_2": "categorical_crossentropy",
	"dense_3": "categorical_crossentropy"
}

model_2.compile(optimizer='adam', loss=losses_2, metrics = [f1])

model_2.fit(X_train_2,
	{"dense_2": y_train3, "dense_3": y_train4},
	validation_data=(X_test_2,
		{"dense_2": y_test3, "dense_3": y_test4}), epochs=50, callbacks = [EarlyStop2], 
            batch_size=24, class_weight=class_weight)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2192 samples, validate on 549 samples
Epoch 1/50
2192/2192 [==============================] - 193s 88ms/step - loss: 1.4299 - dense_2_loss: 0.4729 - dense_3_loss: 0.9570 - dense_2_f1: 0.1159 - dense_3_f1: 0.4616 - val_loss: 4.6128 - val_dense_2_loss: 1.0275 - val_dense_3_loss: 3.5853 - val_dense_2_f1: 0.1002 - val_dense_3_f1: 0.4198
Epoch 2/50
2192/2192 [==============================] - 129s 59ms/step - loss: 1.1494 - dense_2_loss: 0.3646 - dense_3_loss: 0.7848 - dense_2_f1: 0.1053 - dense_3_f1: 0.3969 - val_loss: 1.2933 - val_dense_2_loss: 0.4533 - val_dense_3_loss: 0.8401 - val_dense_2_f1: 0.1239 - val_dense_3_f1: 0.4210
Epoch 3/50
2192/2192 [==============================] - 129s 59ms/step - loss: 1.0406 - dense_2_loss: 0.3267 - dense_3_loss: 0.7140 - dense_2_f1: 0.1040 - dense_3_f1: 0.3360 - val_loss: 1.4019 - val_dense_2_loss: 0.3685 - val_dense_3_loss: 1.0334 - val_dense_2_

In [0]:
testset = []
ans = np.zeros((30, 57), dtype="object")
rootdir = os.path.join(root, "resized_test_dataset/")
for i, name in enumerate(os.listdir(rootdir)):
    ans[i][0]=name
    testset.append(io.imread(os.path.join(rootdir, name))/255)
    
testset = np.array([np.array(testset[i]) for i in range(len(testset))])

In [0]:
y = model.predict(testset)

In [0]:
for i in range(len(y[0])):
    for j in range(len(y[0][0])):
        ans[i][j+4]= 1 if y[0][i][j] >=0.5 else 0

In [0]:
for i in range(len(y[1])):
  ans[i][2] = labels_task2[np.argmax(y[1][i])]

In [0]:
y2 = model_2.predict(testset)

In [0]:
for i in range(len(y2[0])):
  ans[i][1] = labels_task3[np.argmax(y2[0][i])]
  ans[i][3] = labels_task3[np.argmax(y2[1][i])]
 

In [0]:
df = pd.DataFrame(ans,columns=['filename', 'standard', 'task2_class', 'tech_cond'] + labels_task_1)
df.to_csv("ans.csv", index = False)

[]

In [0]:
from PIL import Image

In [0]:
folder = "test"
resizedfolder = "test_resized"

for file in os.listdir(folder):
  if file[:-3]=="jpg":
    im = Image.open(os.path.join(folder,file))
    im.convert("RGB")
    im = im.resize((224,224))
    im.save(os.path.join(resizedfolder,file))

FileNotFoundError: ignored

In [0]:
testset = []
ans = np.zeros((30, 57), dtype="object")
for i, name in enumerate(os.listdir(resizedfolder)):
    ans[i][0]=name
    testset.append(io.imread(os.path.join(resizedfolder, name))/255)
    
testset = np.array([np.array(testset[i]) for i in range(len(testset))])

In [0]:
y = model.predict(testset)
for i in range(len(y[0])):
    for j in range(len(y[0][0])):
        ans[i][j+4]= 1 if y[0][i][j] >=0.5 else 0

for i in range(len(y[1])):
  ans[i][2] = labels_task2[np.argmax(y[1][i])]

y2 = model_2.predict(testset)

for i in range(len(y2[0])):
  ans[i][1] = labels_task3[np.argmax(y2[0][i])]
  ans[i][3] = labels_task3[np.argmax(y2[1][i])]

df = pd.DataFrame(ans,columns=['filename', 'standard', 'task2_class', 'tech_cond'] + labels_task_1)
df.to_csv("finalans.csv", index = False)